In [1]:
import pandas as pd
train=pd.read_csv('train.csv',index_col=0)
test=pd.read_csv('test.csv',index_col=0)

In [2]:
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)
from sklearn.preprocessing import LabelEncoder

# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)

# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)

In [3]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [4]:
param = {'boosting_type':'gbdt',
                         'objective' : 'binary', 
                         'metric' : 'auc',
                         'learning_rate' : 0.01,
                         'max_depth' : 15,
                         'feature_fraction':0.8,
                         'bagging_fraction': 0.9,
                         'bagging_freq': 8,
                         'lambda_l1': 0.6,
                         'lambda_l2': 0,
        }

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid)

In [6]:
model = lgb.train(param,train_data,valid_sets=[train_data,valid_data],num_boost_round = 10000 ,early_stopping_rounds=200,verbose_eval=25, categorical_feature=attr)
predict=model.predict(test)
test['Attrition']=predict

C:\Users\xueliang.chen\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Age', 'BusinessTravel', 'Department', 'Education', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Over18', 'OverTime']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 200 rounds
[25]	training's auc: 0.922112	valid_1's auc: 0.757558
[50]	training's auc: 0.944135	valid_1's auc: 0.770381
[75]	training's auc: 0.957212	valid_1's auc: 0.788825
[100]	training's auc: 0.966265	valid_1's auc: 0.792317
[125]	training's auc: 0.976136	valid_1's auc: 0.797774
[150]	training's auc: 0.981821	valid_1's auc: 0.800939
[175]	training's auc: 0.986519	valid_1's auc: 0.806177
[200]	training's auc: 0.989761	valid_1's auc: 0.808141
[225]	training's auc: 0.992635	valid_1's auc: 0.812398
[250]	training's auc: 0.994369	valid_1's auc: 0.815781
[275]	training's auc: 0.996165	valid_1's auc: 0.820801
[300]	training's auc: 0.997153	valid_1's auc: 0.821019
[325]	training's auc: 0.997791	valid_1's auc: 0.819382
[350]	training's auc: 0.998563	valid_1's auc: 0.820801
[375]	training's auc: 0.999021	valid_1's auc: 0.82091
[400]	training's auc: 0.999317	valid_1's auc: 0.823748
[425]	training's auc: 0.999524	valid_1's auc: 0.827349
[450]	t

In [7]:
test['Attrition']

user_id
442     0.031402
1091    0.013365
981     0.130556
785     0.026596
1332    0.880303
          ...   
1439    0.013608
481     0.024272
124     0.349841
198     0.007679
1229    0.036693
Name: Attrition, Length: 294, dtype: float64

In [8]:
test[['Attrition']].to_csv('submit_lgb.csv')